# K Nearest Neighbors Model

A simple classification model. This notebook can be used as a template for others.

# Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from seaborn import set_style

set_style("whitegrid")

In [ ]:
# For saving trained models
import joblib

In [ ]:
## import here
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
## import train_test_split
from sklearn.model_selection import train_test_split

# Settings

In [ ]:
data_fp = '../../data/processed_data/raw_wav.npy'

In [ ]:
model_filename = '../../data/trained_models/knn.pkl'

# Load Data

In [ ]:
# Load data
x_data = np.load(data_fp)

In [ ]:
# Simulate y classifications
# TODO: Replace this with actual classifications
rng = np.random.default_rng(seed=12987)
y_data = rng.integers(0, 3, size=len(x_data))

# Train-Test Split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data.copy(), y_data,
                                        shuffle = True,
                                        random_state = 440,
                                        test_size=.2)

# Fit Model

In [ ]:
## make the model object
knn = KNeighborsClassifier(5)

## "fit" the model object
knn.fit(x_train, y_train)


# Assess Model Performance
Do things like test accuracy, etc.

In [ ]:
## predict on the training set
y_pred = knn.predict(x_test)

# Save Trained Model

In [ ]:
# Save the model to disk
joblib.dump(knn, model_filename)